In [4]:
# setting the environment variables, the keys
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

from config import set_environment
# for the keys - as explained early in chapter 2
set_environment()

# Image generation and understanding

## Image generation

In [2]:
DallEAPIWrapper??

Init signature:
DallEAPIWrapper(
    *,
    client: Any = None,
    async_client: Any = None,
    model: str = 'dall-e-2',
    model_kwargs: Dict[str, Any] = None,
    api_key: Optional[pydantic.types.SecretStr] = None,
    base_url: Optional[str] = None,
    organization: Optional[str] = None,
    openai_proxy: str = None,
    timeout: Union[float, Tuple[float, float], Any, NoneType] = None,
    n: int = 1,
    size: str = '1024x1024',
    separator: str = '\n',
    quality: Optional[str] = 'standard',
    max_retries: int = 2,
    default_headers: Optional[Mapping[str, str]] = None,
    default_query: Optional[Mapping[str, object]] = None,
    http_client: Optional[Any] = None,
) -> None
Source:        
class DallEAPIWrapper(BaseModel):
    """Wrapper for OpenAI's DALL-E Image Generator.

    https://platform.openai.com/docs/guides/images/generations?context=node

    Usage instructions:

    1. `pip install openai`
    2. save your OPENAI_API_KEY in an environment variable
    """



In [7]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper

dalle = DallEAPIWrapper(
   model="dall-e-3",  # Options: "dall-e-2" (default) or "dall-e-3"
   size="1024x1024",       # Image dimensions
    quality="standard",     # "standard" or "hd" for DALL-E 3
    n=1                     # Number of images to generate (only for DALL-E 2)
)
image_url = dalle.run("A detailed technical diagram of a quantum computer")
print(image_url)

https://oaidalleapiprodscus.blob.core.windows.net/private/org-v2zpDq7lmzeE7QnOAKN5KtVD/user-1kc8PP1odvcm9qydlgvraAFz/img-DByXPYsdEFE8tHuI0aqw1BWo.png?st=2025-03-12T10%3A39%3A39Z&se=2025-03-12T12%3A39%3A39Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-03-11T23%3A43%3A47Z&ske=2025-03-12T23%3A43%3A47Z&sks=b&skv=2024-08-04&sig=O3iW%2BJvjB/Rqjmjcag9W6N%2B/T4N6eWwmqqLi/KDzZ4A%3D


In [9]:
from langchain_community.llms import Replicate

# Initialize the text-to-image model:
text2image = Replicate(
    model="stability-ai/stable-diffusion-3.5-large",
    model_kwargs={
        "prompt_strength": 0.85,
        "cfg": 4.5,
        "steps": 40,
        "aspect_ratio": "1:1",
        "output_format": "webp",
        "output_quality": 90
    }
)


# Generate an image
image_url = text2image.invoke(
    "A detailed technical diagram of an AI agent"
)

print(image_url)

https://replicate.delivery/xezq/BxTlKeSh7P34baSTtxd3oC880eWO1fs7JiasfUfn3CsliSeFF/out-0.webp


## Image understanding

In [7]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

def analyze_image(image_url: str, question: str) -> str:
    chat = ChatOpenAI(model="gpt-4o-mini", max_tokens=256)

    message = HumanMessage(
        content=[
            {
                "type": "text",
                "text": question
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": image_url,
                    "detail": "auto"
                }
            }
        ]
    )

    response = chat.invoke([message])
    return response.content

# Example usage
image_url = "https://replicate.delivery/yhqm/JGTpV46FdT7dE5RGgs0dy5qgLegwOqoIPPaAvrfOmOGMwkMUA/out-0.png"
questions = [
    "What objects do you see in this image?",
    "What is the overall mood or atmosphere?",
    "Are there any people in the image?"
]

for question in questions:
    print(f"\nQ: {question}")
    print(f"A: {analyze_image(image_url, question)}")


Q: What objects do you see in this image?
A: The image features futuristic vehicles flying above a modern cityscape. There are tall skyscrapers, some with intricate designs and bright lighting, creating a vibrant urban environment. The ground appears to have markings that suggest a road or pathway, enhancing the sense of a bustling, advanced society.

Q: What is the overall mood or atmosphere?
A: The overall mood of the image conveys a futuristic and high-tech atmosphere. The sleek design of the vehicles and the towering skyscrapers suggest a vibrant, dynamic cityscape. The use of neon colors and lighting enhances a sense of innovation and excitement, creating an inviting yet electric ambiance. The scene evokes feelings of progress and possibility, characteristic of a sci-fi environment.

Q: Are there any people in the image?
A: No, there are no people in the image. It depicts futuristic vehicles and a cityscape.


### With Gemini

In [ ]:
import base64
with open("stable-diffusion.png", 'rb') as image_file:
 image_bytes = image_file.read()
 base64_bytes = base64.b64encode(image_bytes).decode("utf-8")


prompt = [
   {"type": "text", "text": "Describe the image: "},
   {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_bytes}"}},
]


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
)
response = llm.invoke([HumanMessage(content=prompt)])
print(response.content)

#### Using Google Cloud Storage

In [ ]:
prompt = [
   {"type": "text", "text": "Describe the video in a few sentences."},
   {"type": "media", "file_uri": video_uri, "mime_type": "video/mp4"},
]


response = llm.invoke([HumanMessage(content=prompt)])
print(response.content)